In [11]:
import cv2
import numpy as np
import mediapipe as mp

# face mesh 모델 로드
mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5)

# 첫 번째 이미지에서 face mesh 좌표값 추출
img1 = cv2.imread('images.jpg')
img1_h, img1_w, _ = img1.shape
results1 = face_mesh.process(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
points1 = []
if results1.multi_face_landmarks:
    for landmark in results1.multi_face_landmarks[0].landmark:
        points1.append([landmark.x * img1_w, landmark.y * img1_h])

# 두 번째 이미지에서 face mesh 좌표값 추출
img2 = cv2.imread('images2.jfif')
img2_h, img2_w, _ = img2.shape
results2 = face_mesh.process(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
points2 = []
if results2.multi_face_landmarks:
    for landmark in results2.multi_face_landmarks[0].landmark:
        points2.append([landmark.x * img2_w, landmark.y * img2_h])

# face mesh 좌표값 대응시키기
M, _ = cv2.estimateAffinePartial2D(np.array(points2), np.array(points1))

# 두 번째 이미지를 첫 번째 이미지의 얼굴 모양으로 변환
result = cv2.warpAffine(img2, M, (img1_w, img1_h))

# 결과 이미지 출력
cv2.imshow("Face Transfer Result", result)
cv2.imshow("1",test_m3(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)))
cv2.imshow("2",test_m3(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)))
cv2.waitKey(0)
cv2.destroyAllWindows()

In [9]:
import cv2
import mediapipe as mp

# 이미지 1에서 face mesh 추출
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5) as face_mesh:
    img1 = cv2.imread('images.jpg')
    img1 = cv2.resize(img1, (640, 480))
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(img1)
    img1 = cv2.cvtColor(img1, cv2.COLOR_RGB2BGR)
    if results.multi_face_landmarks:
        face_landmarks1 = results.multi_face_landmarks[0]
c=0
a = []
for i in face_landmarks1.landmark:
    a.append(i.x)
    c+=1
print(len(a))
# 이미지 2에서 face mesh 추출
# with mp_face_mesh.FaceMesh(static_image_mode=True, max_num_faces=1, min_detection_confidence=0.5) as face_mesh:
#     img2 = cv2.imread('images2.jfif')
#     img2 = cv2.resize(img2, (640, 480))
#     img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2RGB)
#     results = face_mesh.process(img2)
#     img2 = cv2.cvtColor(img2, cv2.COLOR_RGB2BGR)
#     if results.multi_face_landmarks:
#         face_landmarks2 = results.multi_face_landmarks[0]

# # 이미지 1에서 face mesh 제거한 영역과 이미지 2에서 face mesh 영역을 합성
# img1_face = img1.copy()
# img2_face = img2.copy()

# for landmark in face_landmarks1.landmark:
#     x, y = int(landmark.x * img1.shape[1]), int(landmark.y * img1.shape[0])
#     cv2.circle(img1_face, (x, y), 1, (0, 0, 255), -1)

# for landmark in face_landmarks2.landmark:
#     x, y = int(landmark.x * img2.shape[1]), int(landmark.y * img2.shape[0])
#     cv2.circle(img2_face, (x, y), 1, (0, 0, 255), -1)

# img2_face_mask = cv2.cvtColor(img2_face, cv2.COLOR_BGR2GRAY)
# img2_face_mask = cv2.merge([img2_face_mask, img2_face_mask, img2_face_mask])
# img1_bg = cv2.bitwise_and(img1, img2_face_mask)
# img2_fg = cv2.bitwise_and(img2, img2_face_mask)
# dst = cv2.add(img1_bg, img2_fg)

# # 합성 결과 출력
# cv2.imshow('Result', dst)
# cv2.waitKey(0)


468


In [3]:
import cv2
import mediapipe as mp
import numpy as np

mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# 이미지 파일 로드
img1 = cv2.imread('images.jpg')
img2 = cv2.imread('images2.jfif')

# face mesh 모델 초기화
with mp_face_mesh.FaceMesh() as face_mesh:

    # 이미지 1에서 face mesh 좌표값 추출
    results1 = face_mesh.process(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    landmarks1 = results1.multi_face_landmarks[0].landmark
    
    # 이미지 2에서 face mesh 좌표값 추출
    results2 = face_mesh.process(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    landmarks2 = results2.multi_face_landmarks[0].landmark
    
    # 좌표값 배열 생성
    landmarks1_arr = np.array([(lmk.x, lmk.y, lmk.z) for lmk in landmarks1])
    landmarks2_arr = np.array([(lmk.x, lmk.y, lmk.z) for lmk in landmarks2])
    
    # affine transformation matrix 계산
    _,transformation_matrix = cv2.estimateAffine3D(landmarks1_arr, landmarks2_arr, ransacThreshold=3.0)
    
    # 이미지 1에서 face mesh 추출 후, 이미지 2의 좌표로 변환
    img1_landmarks = np.concatenate([landmarks1_arr, np.ones((len(landmarks1_arr), 1))], axis=1)
    img2_landmarks = np.dot(transformation_matrix, img1_landmarks.T).T[:,:-1]
    
    # 이미지 1에서 face mesh 제거
    img1_gray = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(img1_gray)
    points = cv2.convexHull(np.array([(lmk[0], lmk[1]) for lmk in img2_landmarks], dtype=np.int32))
    cv2.fillConvexPoly(mask, points, (255, 255, 255))
    img1_face = cv2.bitwise_and(img1, img1, mask=mask)
    
    # 이미지 2에서 face mesh 영역 추출
    img2_gray = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)
    _, thresh = cv2.threshold(img2_gray, 1, 255, cv2.THRESH_BINARY_INV)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    img2_face = np.zeros_like(img2)
    cv2.drawContours(img2_face, contours, -1, (255, 255, 255), -1)
    
    # 이미지 1에서 face mesh 제거한 영역과 이미지 2에서 face mesh 영역을 합성
    img2_face_mask = cv2.cvtColor(img2_face, cv2.COLOR_BGR2GRAY)
    img2_face_mask = cv2.merge([img2_face_mask, img2_face_mask, img2_face_mask])
    img1_bg = cv2.bitwise_and(img1, img1_face)
    img2_fg = cv2.bitwise_and(img2, img2_face_mask)
    dst = cv2.add(img1_face, img2_fg)
    
    # 합성 결과 출력
    cv2.imshow('Result', dst)
    cv2.waitKey(0)


ValueError: too many values to unpack (expected 2)

In [4]:
import cv2
import mediapipe as mp
import numpy as np

mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh

# 이미지 로드
img1 = cv2.imread('images.jpg')
img2 = cv2.imread('images2.jfif')

# face mesh 모델 로드
with mp_face_mesh.FaceMesh() as face_mesh:
    # 이미지 1에서 face mesh 추출
    results1 = face_mesh.process(cv2.cvtColor(img1, cv2.COLOR_BGR2RGB))
    # 좌표 추출
    landmarks1 = results1.multi_face_landmarks[0].landmark
    # 좌표를 numpy 배열로 변환
    landmarks1_arr = np.array([(lmk.x, lmk.y, lmk.z) for lmk in landmarks1])

    # 이미지 2에서 face mesh 추출
    results2 = face_mesh.process(cv2.cvtColor(img2, cv2.COLOR_BGR2RGB))
    # 좌표 추출
    landmarks2 = results2.multi_face_landmarks[0].landmark
    # 좌표를 numpy 배열로 변환
    landmarks2_arr = np.array([(lmk.x, lmk.y, lmk.z) for lmk in landmarks2])

    # affine transformation matrix 계산
    _, transformation_matrix = cv2.estimateAffine3D(landmarks1_arr, landmarks2_arr, ransacThreshold=3.0)

    # 이미지 1에서 face mesh 추출 후, 이미지 2의 좌표로 변환
    img1_landmarks = np.concatenate([landmarks1_arr, np.ones((len(landmarks1_arr), 1))], axis=1)
    img2_landmarks = np.concatenate([landmarks2_arr, np.ones((len(landmarks2_arr), 1))], axis=1)

    img1_landmarks_trans = (transformation_matrix @ img1_landmarks.T).T[:, :3]

    # 변환된 좌표를 이미지 2에 그리기
    img2_copy = np.copy(img2)
    for landmark in img1_landmarks_trans.astype(np.int32):
        cv2.circle(img2_copy, tuple(landmark[:2]), 1, (0, 255, 0), -1)

    # 결과 이미지 출력
    cv2.imshow('result', img2_copy)
    cv2.waitKey(0)
    cv2.destroyAllWindows()


ValueError: too many values to unpack (expected 2)